In [ ]:
import tkinter as tk
from tkinter import messagebox
import random
import copy

# Size of the Sudoku grid
SIZE = 9
CELL_SIZE = 10  # Size of each cell

# Create the Sudoku GUI application
class SudokuGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sudoku Solver")
        self.board = self.generate_random_sudoku()
        self.solution = copy.deepcopy(self.board)
        self.solve_sudoku(self.solution)

        # Create the GUI grid of Entry widgets
        self.entries = [[None for _ in range(SIZE)] for _ in range(SIZE)]
        self.create_grid()

        # Create buttons
        self.start_button = tk.Button(self.root, text="Start New Game", command=self.start_new_game, bg="#2196F3", fg="white", font=("Arial", 14))
        self.start_button.grid(row=SIZE, column=0, columnspan=SIZE//2, pady=10)

        self.restart_button = tk.Button(self.root, text="Restart Game", command=self.restart_game, bg="#1E88E5", fg="white", font=("Arial", 14))
        self.restart_button.grid(row=SIZE, column=SIZE//2, columnspan=SIZE//2, pady=10)

        self.hint_button1 = tk.Button(self.root, text="Hint 1", command=lambda: self.provide_hint(1), bg="#4CAF50", fg="white", font=("Arial", 12))
        self.hint_button1.grid(row=SIZE+1, column=0, pady=5)

        self.hint_button2 = tk.Button(self.root, text="Hint 2", command=lambda: self.provide_hint(2), bg="#4CAF50", fg="white", font=("Arial", 12))
        self.hint_button2.grid(row=SIZE+1, column=1, pady=5)

        self.hint_button3 = tk.Button(self.root, text="Hint 3", command=lambda: self.provide_hint(3), bg="#4CAF50", fg="white", font=("Arial", 12))
        self.hint_button3.grid(row=SIZE+1, column=2, pady=5)

    # Create the Sudoku grid on the GUI
    def create_grid(self):
        for i in range(SIZE):
            for j in range(SIZE):
                entry = tk.Entry(self.root, width=4, font=("Arial", 18), justify="center", bd=2, relief="solid")
                entry.grid(row=i, column=j, padx=1, pady=1, sticky="nsew", ipadx=CELL_SIZE//2, ipady=CELL_SIZE//2)
                entry.bind("<KeyRelease>", self.on_key_release)  # Bind to key release event
                entry.config(bg="white", fg="black")
                if self.board[i][j] != 0:
                    entry.insert(0, str(self.board[i][j]))
                    entry.config(state="disabled", disabledbackground="#B3CDE0", disabledforeground="black")
                self.entries[i][j] = entry

        # Configure row and column weights for resizing
        for i in range(SIZE):
            self.root.grid_rowconfigure(i, weight=1)
            self.root.grid_columnconfigure(i, weight=1)

        # Add borders to 3x3 sub-grids
        self.add_grid_borders()

    # Add borders to 3x3 sub-grids
    def add_grid_borders(self):
        for i in range(0, SIZE, 3):
            for j in range(0, SIZE, 3):
                for x in range(3):
                    for y in range(3):
                        cell = self.entries[i + x][j + y]
                        cell.config(borderwidth=2, relief="solid", bg="white")
                        
                # Add thicker borders around 3x3 boxes
                for x in range(3):
                    self.entries[i + x][j].config(highlightbackground="black", highlightthickness=2)
                    self.entries[i + x][j + 2].config(highlightbackground="black", highlightthickness=2)
                    self.entries[i][j + x].config(highlightbackground="black", highlightthickness=2)
                    self.entries[i + 2][j + x].config(highlightbackground="black", highlightthickness=2)

    # Provide a hint by revealing a random empty cell
    def provide_hint(self, hint_number):
        empty_cells = [(i, j) for i in range(SIZE) for j in range(SIZE) if self.board[i][j] == 0]
        if not empty_cells:
            messagebox.showinfo("Hint", "No empty cells available for hints.")
            return

        random.shuffle(empty_cells)
        for i, j in empty_cells:
            if self.entries[i][j].get() == "":
                entry = self.entries[i][j]
                entry.insert(0, str(self.solution[i][j]))
                entry.config(fg="blue", bg="#e0f7fa")
                entry.config(state="disabled")
                break

    # Handle key release events for immediate feedback
    def on_key_release(self, event):
        row, col = self.get_entry_position(event.widget)
        if row is not None and col is not None:
            entry = self.entries[row][col]
            try:
                user_value = int(entry.get())
                if self.is_valid_move(self.board, user_value, row, col):
                    if user_value == self.solution[row][col]:
                        entry.config(fg="green")
                        entry.config(bg="#e0f7fa")  # Light cyan background for correct
                    else:
                        entry.config(fg="red")
                        entry.config(bg="#ffebee")  # Light pink background for incorrect
                else:
                    entry.config(fg="red")
                    entry.config(bg="#ffebee")  # Light pink background for invalid
                self.check_win()
            except ValueError:
                entry.config(fg="red")
                entry.config(bg="#ffebee")  # Light pink background for invalid

    # Start a new Sudoku game
    def start_new_game(self):
        self.board = self.generate_random_sudoku()
        self.solution = copy.deepcopy(self.board)
        self.solve_sudoku(self.solution)
        self.update_grid()

    # Restart the current game
    def restart_game(self):
        self.update_grid()

    # Update the GUI grid with a new puzzle
    def update_grid(self):
        for i in range(SIZE):
            for j in range(SIZE):
                entry = self.entries[i][j]
                if self.board[i][j] != 0:
                    entry.config(state="normal")
                    entry.delete(0, tk.END)
                    entry.insert(0, str(self.board[i][j]))
                    entry.config(disabledbackground="#B3CDE0", disabledforeground="black", state="disabled")
                    entry.config(bg="#B3CDE0")
                else:
                    entry.config(state="normal", fg="black", bg="white")
                    entry.delete(0, tk.END)

        # Update the grid cell colors for 3x3 sub-grids
        self.add_grid_borders()

    # Get the row and column of an entry widget
    def get_entry_position(self, widget):
        for row in range(SIZE):
            for col in range(SIZE):
                if self.entries[row][col] == widget:
                    return row, col
        return None, None

    # Generate a random Sudoku board by solving a full Sudoku and then removing numbers
    def generate_random_sudoku(self):
        # Generate a solved Sudoku board
        board = [[0 for _ in range(SIZE)] for _ in range(SIZE)]
        self.solve_sudoku(board)

        # Remove random cells to create the puzzle
        num_remove = random.randint(40, 55)
        for _ in range(num_remove):
            row = random.randint(0, SIZE - 1)
            col = random.randint(0, SIZE - 1)
            while board[row][col] == 0:
                row = random.randint(0, SIZE - 1)
                col = random.randint(0, SIZE - 1)
            board[row][col] = 0

        return board

    # Solve the Sudoku puzzle using backtracking
    def solve_sudoku(self, board):
        empty = self.find_empty_cell(board)
        if not empty:
            return True
        row, col = empty

        for num in range(1, SIZE + 1):
            if self.is_valid_move(board, num, row, col):
                board[row][col] = num

                if self.solve_sudoku(board):
                    return True

                board[row][col] = 0

        return False

    # Find the next empty cell (0) in the board
    def find_empty_cell(self, board):
        for i in range(SIZE):
            for j in range(SIZE):
                if board[i][j] == 0:
                    return (i, j)
        return None

    # Check if placing the number is valid according to Sudoku rules
    def is_valid_move(self, board, num, row, col):
        # Check the row
        if num in board[row]:
            return False

        # Check the column
        if num in [board[i][col] for i in range(SIZE)]:
            return False

        # Check the 3x3 sub-grid
        box_start_row, box_start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(box_start_row, box_start_row + 3):
            for j in range(box_start_col, box_start_col + 3):
                if board[i][j] == num:
                    return False

        return True

    # Check if the user has won the game
    def check_win(self):
        for i in range(SIZE):
            for j in range(SIZE):
                if self.entries[i][j].get() != str(self.solution[i][j]):
                    return
        messagebox.showinfo("Congratulations!", "You have solved the Sudoku!")

# Main function to run the Sudoku GUI
if __name__ == "__main__":
    root = tk.Tk()
    sudoku_gui = SudokuGUI(root)
    root.mainloop()